In [3]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session


# Options
Price_Source = 'close' # close or ohlc
Layer_Number = 3 # 2,3 or 4
Activation_Func = 'leaky_relu' # relu or leaky_relu


# Use CNN to capture local temporal dependency of data in risk prediction or other related tasks.
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


if Price_Source == 'close':
    input_file_name = 'bitcoin2016-2020_close.h5'
    
with h5py.File(''.join([input_file_name]), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


file_path = Price_Source + '_CNN_' + str(Layer_Number) + '_' + Activation_Func
output_file_name='bitcoin2016-2020_' + Price_Source + '_CNN_' + str(Layer_Number) + '_' + Activation_Func


step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]
epochs = 100


#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]


#build model
model = Sequential()

if Activation_Func == 'relu':
    
    if Layer_Number == 2:
        # 2 layers
        model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
        model.add(Dropout(0.5))
        model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

    elif Layer_Number == 3:
        # 3 Layers
        model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=24))
        model.add(Dropout(0.5))
        model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=20))
        model.add(Dropout(0.5))
        model.add(Conv1D( strides=2, filters=nb_features, kernel_size=18))

    elif Layer_Number == 4:
        # 4 layers
        model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
        model.add(Dropout(0.5))
        model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
        model.add(Dropout(0.5))
        model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=2, filters=nb_features, kernel_size=2))
        
elif Activation_Func == 'leaky_relu':
    
    if Layer_Number == 2:
        # 2 layers
        model.add(Conv1D(input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=4, filters=nb_features, kernel_size=16))

    elif Layer_Number == 3:
        # 3 Layers
        model.add(Conv1D(input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=24))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=2, filters=8, kernel_size=20))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=2, filters=nb_features, kernel_size=18))

    elif Layer_Number == 4:
        # 4 layers
        model.add(Conv1D(input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=26))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=2, filters=8, kernel_size=22))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=1, filters=8, kernel_size=18))
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
        model.add(Conv1D(strides=1, filters=nb_features, kernel_size=16))
    

model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger('callbacks/'+output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+file_path+'/'+output_file_name+'-{epoch:02d}.hdf5', monitor='val_loss', verbose=1,mode='min')])



C:\Users\Kerry\Anaconda3\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Train on 1293 samples, validate on 324 samples
Epoch 1/100
1293/1293 [==============================] - 1s 700us/step - loss: 0.0155 - val_loss: 3.2949e-04

Epoch 00001: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-01.hdf5
Epoch 2/100
1293/1293 [==============================] - 1s 653us/step - loss: 0.0030 - val_loss: 1.6614e-04

Epoch 00002: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-02.hdf5
Epoch 3/100
1293/1293 [==============================] - 1s 629us/step - loss: 0.0022 - val_loss: 1.8422e-04

Epoch 00003: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-03.hdf5
Epoch 4/100
1293/1293 [==============================] - 1s 568us/step - loss: 0.0028 - val_loss: 1.8241e-04

Epoch 00004: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-04.hdf5
Epoch 5/100
1293/1293 [==============================] - 1s 545us/step - loss: 0.

Epoch 38/100
1293/1293 [==============================] - 1s 543us/step - loss: 7.6523e-04 - val_loss: 1.5044e-04

Epoch 00038: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-38.hdf5
Epoch 39/100
1293/1293 [==============================] - 1s 556us/step - loss: 9.5808e-04 - val_loss: 3.8936e-04

Epoch 00039: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-39.hdf5
Epoch 40/100
1293/1293 [==============================] - 1s 522us/step - loss: 9.4061e-04 - val_loss: 6.9409e-04

Epoch 00040: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-40.hdf5
Epoch 41/100
1293/1293 [==============================] - 1s 575us/step - loss: 8.1604e-04 - val_loss: 1.5328e-04

Epoch 00041: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-41.hdf5
Epoch 42/100
1293/1293 [==============================] - 1s 561us/step - loss: 8.1082e-04 - val_loss: 0.001

1293/1293 [==============================] - 1s 547us/step - loss: 8.4297e-04 - val_loss: 7.5009e-04

Epoch 00075: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-75.hdf5
Epoch 76/100
1293/1293 [==============================] - 1s 511us/step - loss: 7.1406e-04 - val_loss: 1.7510e-04

Epoch 00076: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-76.hdf5
Epoch 77/100
1293/1293 [==============================] - 1s 530us/step - loss: 7.5758e-04 - val_loss: 1.6224e-04

Epoch 00077: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-77.hdf5
Epoch 78/100
1293/1293 [==============================] - 1s 530us/step - loss: 6.7083e-04 - val_loss: 1.4731e-04

Epoch 00078: saving model to weights/close_CNN_3_leaky_relu/bitcoin2016-2020_close_CNN_3_leaky_relu-78.hdf5
Epoch 79/100
1293/1293 [==============================] - 1s 523us/step - loss: 6.4888e-04 - val_loss: 2.3664e-04

Epoch 